In [25]:
from datetime import date
from datetime import timedelta
from nsepy import get_history
from nsepy.derivatives import get_expiry_date
import numpy as np
import pprint
import math
from nsetools import Nse
nseObj = Nse()
nifty = get_history(symbol='NIFTY 50',start=date(2020,11,18),end=date(2021,11,18),index=True)
pprint.pprint(nifty)

                Open      High       Low     Close       Volume      Turnover
Date                                                                         
2020-11-18  12860.10  12948.85  12819.35  12938.25  731321438.0  3.985645e+11
2020-11-19  12839.50  12963.00  12745.75  12771.70  768751782.0  4.474708e+11
2020-11-20  12813.40  12892.45  12730.25  12859.05  663997267.0  4.299215e+11
2020-11-23  12960.30  12968.85  12825.70  12926.45  633117401.0  4.203138e+11
2020-11-24  13002.60  13079.10  12978.00  13055.15  571864976.0  3.709423e+11
...              ...       ...       ...       ...          ...           ...
2021-11-11  17967.45  17971.35  17798.20  17873.60  232121151.0  2.007478e+11
2021-11-12  17977.60  18123.00  17905.90  18102.75  249111898.0  2.216433e+11
2021-11-15  18140.95  18210.15  18071.30  18109.45  280428633.0  1.909960e+11
2021-11-16  18127.05  18132.65  17958.80  17999.20  267431829.0  2.410167e+11
2021-11-17  17939.35  18022.65  17879.25  17898.65  295715938.0 

In [26]:
niftynew = nifty.filter(['Date','Close'], axis=1)
count = 0
delta = timedelta(days=1)

for ind in niftynew.index:
    # print(ind)
    if count == 0 :
        niftynew.loc[ind].at['Percentage Change'] = 0
        oldind = ind
        count = count + 1
    else :
        difference = abs(niftynew.loc[ind].at['Close'] - niftynew.loc[oldind].at['Close'])
        percentage = (difference/niftynew.at[oldind, 'Close'])*100
        # print(percentage)
        niftynew.at[ind, 'Percentage Change'] = percentage
        count = count + 1
        oldind = ind

pprint.pprint(niftynew)

               Close  Percentage Change
Date                                   
2020-11-18  12938.25                NaN
2020-11-19  12771.70           1.287268
2020-11-20  12859.05           0.683934
2020-11-23  12926.45           0.524144
2020-11-24  13055.15           0.995633
...              ...                ...
2021-11-11  17873.60           0.797016
2021-11-12  18102.75           1.282058
2021-11-15  18109.45           0.037011
2021-11-16  17999.20           0.608798
2021-11-17  17898.65           0.558636

[247 rows x 2 columns]


In [27]:
stddf = niftynew.std(axis = 0, skipna = True)

meandf = niftynew.mean(axis = 0, skipna= True)

print("Standard Deviation")
print(meandf)
print("Mean")
print(stddf)


Standard Deviation
Close                15556.289474
Percentage Change        0.712286
dtype: float64
Mean
Close                1455.598494
Percentage Change       0.651626
dtype: float64


In [28]:
today = date.today()
expiryDate = date(2021,11,25)
dateDifference = np.busday_count(today, expiryDate)
mean = dateDifference * meandf.loc["Percentage Change"]
print(mean)
std = math.sqrt(dateDifference) * stddf.loc["Percentage Change"]
print(std)

3.561431030541573
1.4570790790263501


In [29]:
dicTodaysvalue = nseObj.get_index_quote("nifty 50")
print(dicTodaysvalue.get("lastPrice"))
todaysValue = dicTodaysvalue.get("lastPrice")
ExpectedUpperValue = todaysValue*(1+(mean/100))
ExpectedLowerValue = todaysValue*(1-(mean/100))
print("ExpectedLowerValue " + str(ExpectedLowerValue))
print("ExpectedUpperValue " + str(ExpectedUpperValue))
FirstSDUpperValue = todaysValue*(1+((mean+std)/100))
FirstSDLowerValue = todaysValue*(1-((mean+std)/100))
print("FirstSDLowerValue " + str(FirstSDLowerValue))
print("FirstSDUpperValue " + str(FirstSDUpperValue))
SecondSDUpperValue = todaysValue*(1+((mean+(2*std))/100))
SecondSDLowerValue = todaysValue*(1-((mean+(2*std))/100))
print("SecondSDLowerValue " + str(SecondSDLowerValue))
print("SecondSDUpperValue " + str(SecondSDUpperValue))

17775.9
ExpectedLowerValue 17142.82358144196
ExpectedUpperValue 18408.976418558042
FirstSDLowerValue 16883.814661433316
FirstSDUpperValue 18667.985338566687
SecondSDLowerValue 16624.805741424672
SecondSDUpperValue 18926.99425857533
